## REI602M Machine Learning - Homework 9
### Due: *Monday* 18.3.2019

**Objectives**: The PageRank algorithm, Feedforward neural networks, backpropagation, mini-batch gradient descent.

**Name**: Emil Gauti Friðriksson, **email: ** egf3@hi.is, **collaborators:** (if any)

**Note**: Problems 2 and 3 are taken from Andrew Ng's Machine learning course at Stanford. Several solutions to them have been made available on the web but I trust that you will not look them up and solve the problems on your own (feel free to collaborate though). The third problem will take some time, but when you do complete it you will have acquired a solid understanding on training feedforward neural networks.

1\. [PageRank, 30 points] In this problem you implement a simple version of the PageRank algorithm.
<img src="simple_web.png" width="200">
Construct the normalized connectivity matrix $Q$ for the above graph where nodes represent web pages and directed edges represent hyperlinks, $Q_{ij}=1/N_j$ if there is a link from $j$ to $i$ and $N_j$ is the number of outlinks from page $j$. Let $e$ denote a vector of ones and $d_j=1$ if $N_j=0$ but zero otherwise. Calculate the matrices $P=Q+(1/n)ed^T$ and $A=\alpha P + (1-\alpha)(1/n)ee^T$ for $\alpha=0.85$. Report $Q,P$ and $A$. The ranking $r$ is given by the largest eigenvector of $A$. Compute $r$ by implementing the *power method*, starting from $r^{0}=(1/n)1$ and using $\epsilon=10^{-4}$.

Iterate $k=1,2,\ldots$

$\quad q^{(k)} = Ar^{(k-1)}$

$\quad r^{(k)}=q^{(k)} / ||q^{(k)}||_1$

until $||r^{(k)} - r^{(k-1)}|| < \epsilon $.

Report the rank of each of the nodes in descending order.

In [50]:
import numpy as np
alpha=0.85
n=8#fjöldi hnúta
Q = np.zeros((n,n))
# Create the dictionary with graph elements
graph = { "a" : ["e","c"],
          "b" : ["a"],
          "c" : ["b", "d"],
          "d" : ["e","b","f"],
          "e" : ["a"],
          "f" : [],
          "g" : ["h"],
          "h" : ["g"]
         }
abc = ['a','b','c','d','e','f','g','h']

for i,val1 in enumerate(abc):
    for j,val2 in enumerate(abc):
        if val1 in graph[val2]:
            Q[i,j] = 1/len(graph[val2])

d = np.array([0,0,0,0,0,1,0,0])
e = np.ones(n)

P = Q + (1/n)*e*d.T
A = alpha*P+(1-alpha)*(1/n)*e*e.T

def r_func(A,n):
    r = 1/n*np.ones(n)
    eps = 10e-4
    error = 1
    while error>eps:
        q = A@r
        r_new=q/np.linalg.norm(q)
        error = np.linalg.norm(r-r_new)
        r=r_new
    return(r)

r = r_func(A,n)

np.set_printoptions(linewidth=400)
print('-'*100)
print('Q:')
print(Q)
print('-'*100)
print('A:')
print(A)
print('-'*100)
print('P:')
print(P)
print('-'*100)
print('-'*100)

print('{:^10s} {:^10s}'.format('node','r'))
print('-'*22)
for i in range(n):
    print('{:^10s} {:^10f}'.format(abc[np.argsort(r)[::-1][i]],r[np.argsort(r)[::-1][i]] ))


----------------------------------------------------------------------------------------------------
Q:
[[0.         1.         0.         0.         1.         0.         0.         0.        ]
 [0.         0.         0.5        0.33333333 0.         0.         0.         0.        ]
 [0.5        0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.5        0.         0.         0.         0.         0.        ]
 [0.5        0.         0.         0.33333333 0.         0.         0.         0.        ]
 [0.         0.         0.         0.33333333 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         0.         0.         1.         0.        ]]
----------------------------------------------------------------------------------------------------
A:
[[0.01875    0.86875    0.01875    0.01875    0.86875    0.125 

2\. [Neural networks, 30 points] Let $X=\{(x^{(1)}, \ldots, x^{(n)}\}$ be a dataset of $n$ samples with 2-dimensional features, i.e. $x^{(i)} \in R^2$, that are classified into two categories with labels $ y \in \{0,1\}$. The data set is shown below.

<img src="prob2data.png" width="300">

The examples in class 1 are marked as "x" and examples in class 0 are marked as "0". The task is to perform binary classification using a simple neural network with the architecure shown below.

<img src="prob2network.png" width="200">

Denote the two features $x_1$ and $x_2$, the three neurons in the hidden layer, $h_1,~h_2,~h_3$, and the output neuron as $o$. Let the weight from $x_i$ to $h_j$ be $w_{i,j}^{[1]}$ for $i\in\{1,2\},~j \in \{1,2,3\}$, and the weight from $h_j$ to $o$ be $w_j^{[2]}$. Finally denote the intercept weight for $h_j$ as $w_{0,j}^{[1]}$ and the intercept weight for $o$ as $w_{0}^{[2]}$. The averaged squared loss will be used as a loss function, instead of the usual negative log-likelihood,
$$
\ell = \frac{1}{n} \sum_{i=1}^n(o^{(i)} - y^{(i)})^2
$$
where $o^{(i)}$ is the result of the output neuron for example $i$.

a) [10 points] Suppose that the sigmoid function is used as the activation function for $h_1,~h_2,~h_3$ and $o$. What is the gradient descent update to $w_{1,2}^{[1]}$ assuming that a learning rate of $\alpha$ is used? Your answer should be written in terms of $x^{(i)},~o^{(i)},y^{(i)}$ and the weights.

b) [10 points] Now, suppose instead of using the sigmoid function for the activation function for $h_1,~h_2,h_3$ and $o$, the step function $f(x)$ is used instead, defined as
$$
f(x) = \left\{ \begin{array}{cc} 1&x \geq 0 \\ 0&x < 0 \end{array} \right.
$$

What is one set of weights that would allow the neural network to classify this dataset with 100% accuracy? Please specify a value for the weights in the order given below and explain your reasoning.
$$
w_{0,1}^{[1]} = ?,~ w_{1,1}^{[1]} = ?,~w_{2,1}^{[1]} = ?
$$
$$
w_{0,2}^{[1]} = ?,~ w_{1,2}^{[1]} = ?,~w_{2,2}^{[1]} = ?
$$
$$
w_{0,3}^{[1]} = ?,~ w_{1,3}^{[1]} = ?,~w_{2,3}^{[1]} = ?
$$
$$
w_{0}^{[2]} = ?,~ w_{1}^{[2]} = ?,~w_{2}^{[2]} = ?,~~w_{3}^{[2]} = ?
$$

*Hint*: The are three sides to a triangle, and there are three neurons in the hidden layer.

c) [10 points] Let the activation functions for $h_1,~h_2,~h_3$ be the linear function $f(x)=x$ and the activation function for $o$ be the same step function as before. Is there a specific set of weights that will make the loss 0? If yes, please explicitly state a value for every weight. If not, please explain your reasoning.

3\. [Neural network classification of the MNIST data set, 40 points] In this problem, you will implement a simple neural network to classify grayscale images of handwritten digits (0 - 9) from the MNIST dataset. The dataset contains 50,000 training images, 10,000 validation images and 10,000 testing images of handwritten digits, 0 - 9, represented as a vector of $28 \times 28 = 784$ elements. Download the dataset from https://notendur.hi.is/steinng/kennsla/2019/ml/data/hw9_mnist.zip

To start, you will implement a neural network with a single hidden layer and cross entropy loss, and train it with the MNIST data set. Use the sigmoid function as activation for the hidden layer, and the softmax function (see comments below) for the output layer.

<img src="prob3network.png" width="400">

The cross entropy loss for a single example $(x, y)$ and $K$ classes is
$$
CE(y,\hat{y}) = -\sum_{k=1}^K y_k \log \hat{y}_k
$$
where $\hat{y}\in R^k$ is the vector of softmax outputs from the model for training example $x$ and $y \in R^K$ is the ground truth vector for training example $x$ such that $y=[0,\ldots,0,1,0,\ldots,0]^T$ contains a single 1 at the position of the correct class (this is sometimes referred to as a "one-hot" representation).

For $n$ training examples, the cross entropy loss is averaged over the $n$ examples,
$$
J(W^{[1]}, W^{[2]}, b^{[1]}, b^{[2]}) = \frac{1}{n} \sum_{i=1}^n CE(y^{(i)}, \hat{y}^{(i)})
= -\frac{1}{n} \sum_{i=1}^n \sum_{k=1}^k y_k^{(i)} \log \hat{y_k}{(i)}.
$$

The starter code below already converts labels into one hot representations for you.

Instead of batch gradient descent or stochastic gradient descent, common practice is to use mini-batch gradient descent for deep learning tasks. In this case, the cost function is defined as follows,
$$
J_{MB} = \frac{1}{B} \sum_{i=1}^B CE(y^{(i)},\hat{y}^{(i)})
$$
where $B$ is the number of training examples in the mini-batch.

a) [25 points] Implement both forward-propagation and back-propagation for the above loss function. Initialize the weights of the network by sampling values from a standard normal distribution, e.g. $N(0,0.1)$. Initialize the bias/intercept term to 0. Set the number of hidden units to be 300, and learning rate to be 5. Set $B = 1000$ (mini batch size). This means that you train with 1,000 examples in each iteration. Therefore, for each epoch, 50 iterations are needed to cover the entire training data. The images are pre-shuffled. So you don’t need to randomly sample the data, and can just create mini-batches sequentially.
Train the model with mini-batch gradient descent as described above. Run the training for 30 epochs. At the end of each epoch, calculate the value of loss function averaged over the entire training set, and plot it ($y$-axis) against the number of epochs ($x$-axis). In the same image, plot the value of the loss function averaged over the validation set, and plot it against the number of epochs.
Similarly, in a new image, plot the accuracy (on $y$-axis) over the training set, measured as the fraction of correctly classified examples, versus the number of epochs ($x$-axis). In the same image, also plot the accuracy over the validation set versus number of epochs.
Also, at the end of 30 epochs, save the learnt parameters (i.e all the weights and biases) into a file, so that next time you can directly initialize the parameters with these values from the file, rather than re-training all over. You do NOT need to hand in these parameters.

b) [10 points] Now add a regularization term to your cross entropy loss. The loss function
will become
$$
J_{MB} = \frac{1}{B} \sum_{i=1}^B CE(y^{(i)},\hat{y}^{(i)}) + \lambda (||W^{[1]}||^2 + ||W^{[2]}||^2)
$$
Be careful not to regularize the bias/intercept term (see section 7.1 in the Deep learning book). Set $\lambda$ to be 0.0001. Implement the regularized version and plot the same figures as part (a). Be careful NOT to include the regularization term to calculate the loss value (regularization should only be used to calculate the gradients). Submit the two new plots obtained.
Compare the plots obtained from the regularized version with the plots obtained from the non-regularized version, and summarize your observations in a couple of sentences. As in the previous part, save the learnt parameters (weights and biases) into a different file so that we can initialize from them next time.

c) [5 points] All this while you should have stayed away from the test data completely.
Now that you have convinced yourself that the model is working as expected, it is finally time to measure the model performance on the test set. Once we measure the test set performance, we report it whatever value it may be, and NOT go back and refine the model any further.

Initialize your model from the parameters saved in part (a) (i.e, the non-regularized model), and evaluate the model performance on the test data. Repeat this using the parameters saved in part (b) (i.e, the regularized model). Report your test accuracy for both regularized model and non-regularized model.

*Hint*: Be sure to vectorize your code as much as possible! Training can be very slow otherwise.

*Comments*:

1) Andrej Karpathy explains why you should understand the backpropagation algorithm: https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b

2) The softmax function is a generalization of the logistic function for $K$ classes. See https://en.wikipedia.org/wiki/Softmax_function

3) When debugging your code, use a subset of the data and start e.g. with a 2-class problem. For information on how to check the gradient calculations and other sanity checks, see http://cs231n.github.io/neural-networks-3/#gradcheck

4) Yann Le Cun reports a 4.7% error rate for a similar network (using mean square error instead of cross-entropy) so you should not be disappointed if your network fails to achieve superhuman performance!

In [51]:
# Starter code and utility functions for problem 2
import numpy as np
import matplotlib.pyplot as plt

def readData(images_file, labels_file):
    x = np.loadtxt(images_file, delimiter=',')
    y = np.loadtxt(labels_file, delimiter=',')
    return x, y

def writeData(fname, X):
    np.savetxt(fname, X, delimiter=',')
    return

def softmax(x):
    """
    Compute softmax function for input. 
    """
    ### YOUR CODE HERE
    ### END YOUR CODE
    return s.T

def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    """
    ### YOUR CODE HERE
    ### END YOUR CODE
    return s

def forward_prop(data, labels, params):
    """
    return hidder layer, output(softmax) layer and loss
    """
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']
  
    ### YOUR CODE HERE
    ### END YOUR CODE
    return h, y, cost

def backward_prop(data, labels, params):
    """
    return gradient of parameters
    """
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']
  
    ### YOUR CODE HERE
    ### END YOUR CODE
  
    grad = {}
    grad['W1'] = gradW1.T
    grad['W2'] = gradW2.T
    grad['b1'] = gradb1.T
    grad['b2'] = gradb2.T
  
    return grad

def nn_train(trainData, trainLabels, devData, devLabels):
    (m, n) = trainData.shape
    m, K = trainLabels.shape
    num_hidden = 300
    learning_rate = 5
    params = {}
  
    ### YOUR CODE HERE
    ### END YOUR CODE
  
    return params

def nn_test(data, labels, params):
    h, output, cost = forward_prop(data, labels, params)
    accuracy = compute_accuracy(output, labels)
    return accuracy

def compute_accuracy(output, labels):
    accuracy = (np.argmax(output,axis=1) == np.argmax(labels,axis=1)).sum() * 1. / labels.shape[0]
    return accuracy

def one_hot_labels(labels):
    one_hot_labels = np.zeros((labels.size, 10))
    one_hot_labels[np.arange(labels.size),labels.astype(int)] = 1
    return one_hot_labels

In [52]:
# Read the data, construct a validation set and do some basic preprocessing
# (this takes a while but you only need to do this once)
np.random.seed(100)
trainData, trainLabels = readData('images_train.csv', 'labels_train.csv')
trainLabels = one_hot_labels(trainLabels)
p = np.random.permutation(60000)
trainData = trainData[p,:]
trainLabels = trainLabels[p,:]

devData = trainData[0:10000,:]
devLabels = trainLabels[0:10000,:]
trainData = trainData[10000:,:]
trainLabels = trainLabels[10000:,:]

mean = np.mean(trainData)
std = np.std(trainData)
trainData = (trainData - mean) / std
devData = (devData - mean) / std

testData, testLabels = readData('images_test.csv', 'labels_test.csv')
testLabels = one_hot_labels(testLabels)
testData = (testData - mean) / std

OSError: images_train.csv not found.

In [ ]:
# Check that the data was read correctly
import matplotlib.pyplot as plt
print(trainData[10].shape)
plt.imshow(trainData[10].reshape(28,28),cmap="gray") # Should be a "2"
plt.show()

In [ ]:
# You need to fill in the code above before executing this part
params = nn_train(trainData, trainLabels, devData, devLabels)

readyForTesting = True
if readyForTesting:
    accuracy = nn_test(testData, testLabels, params)
print('Test accuracy: %f' % accuracy)